In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test  = pd.read_csv('../input/test.csv')

df_train.describe()

In [ ]:
def is_num(x):
    x = df_train[x]
    try:
        if x.dtype == np.int64 or x.dtype == np.float64:
            return True
        else:
            return False
    except AttributeError:
        return False
    
def is_complete(x):
    if x == 'SalePrice': return False
    return pd.isnull(df_train[x]).sum() + pd.isnull(df_test[x]).sum() == 0

print(list(map(is_num, list(df_train))))

In [ ]:
categorical = list(filter(lambda x: not is_num(x) and is_complete(x), list(df_train)))
continuous = list(filter(lambda x: is_num(x) and is_complete(x), list(df_train)))
#continuous = list(filter(lambda x: not x == 'SalePrice', continuous))
columns = list(df_train)

In [ ]:
def input_fn(df, train=True):

    continuous_cols = {k: tf.constant(df[k].values)
                        for k in continuous}

    categorical_cols = {k: tf.SparseTensor(
        indices=[[i, 0] for i in range(df[k].size)],
        values=df[k].values,
        dense_shape=[df[k].size, 1])
                        for k in categorical}

   
    feature_cols = continuous_cols.copy()
    feature_cols.update(categorical_cols)

    label = tf.transpose(tf.constant(df['SalePrice'].values))

    return feature_cols, label

def train_input_fn():
    return input_fn(df_train)

def eval_input_fn():
    return input_fn(df_test)


In [ ]:
#print(categorical)
print(df_train['SalePrice'].values)

In [ ]:
print(list(set(df_train['MSZoning'])))

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
layers = []

In [ ]:
for x in categorical:
    print(x)
    print(set(df_train[x]))
    print(' ')

In [ ]:
for x in categorical:
    layers.append(tf.contrib.layers.sparse_column_with_keys(
         column_name=x, keys=list(set(df_train[x]))))

In [ ]:
print(continuous)

In [ ]:
for x in continuous:
    layers.append(tf.contrib.layers.real_valued_column(x))

In [ ]:
print(df_train.shape)
print(df_train.dropna().shape)

print(pd.isnull(df_train).sum() > 0)
print(list(filter(lambda x: , list(df_train))))


In [ ]:

model_dir = "."
m = tf.contrib.learn.LinearRegressor(feature_columns=layers,
    optimizer=tf.train.FtrlOptimizer(
        learning_rate=0.1,
        l1_regularization_strength=1.0,
        l2_regularization_strength=1.0),
    model_dir=model_dir)


In [ ]:
m.fit(input_fn=train_input_fn, steps=200)


In [ ]:
results = m.predict(input_fn=eval_input_fn)
